In [1]:
print("""
@File         : 3-aggregations.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2025-08-10 16:05:13
@Email        : cuixuanstephen@gmail.com
@Description  : Aggregating data
""")


@File         : 3-aggregations.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2025-08-10 16:05:13
@Email        : cuixuanstephen@gmail.com
@Description  : Aggregating data



In [2]:
import numpy as np
import pandas as pd

fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(
        x['volume'], bins=3, labels=["low", "med", "high"]
    )
)

weather = pd.read_csv('data/weather_by_station.csv',
                      index_col='date', parse_dates=True)

In [3]:
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.0,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"


In [4]:
pd.set_option('display.float_format', lambda x: "%.2f" % x)

#### Summarizing DataFrames

In [5]:
fb.agg({
    'open': 'mean', 'high': 'max', 'low': 'min', 'close': 'mean', 'volume': "sum"
})

open            171.45
high            218.62
low             123.02
close           171.51
volume   6949682394.00
dtype: float64

In [8]:
weather.query("station == 'GHCND:USW00094728'").pivot(
    columns='datatype', values='value'
).sum()

datatype
AWND     569.90
PRCP    1665.30
SNOW    1007.00
SNWD    1670.00
TMAX    6215.00
TMIN    3506.50
WDF2   55740.00
WDF5   56560.00
WSF2    1623.30
WSF5    2661.10
WT01     176.00
WT02      10.00
WT03      25.00
WT06       1.00
WT08      61.00
dtype: float64

In [9]:
fb.agg({
    'open': 'mean', 
    'high': ['max', 'min'], 
    'low': ['min', 'max'], 
    'close': 'mean',
})

,open,high,low,close
mean,171.45,NaN,NaN,171.51
max,NaN,218.62,214.27,NaN
min,NaN,129.74,123.02,NaN


#### Aggregating by group

In [11]:
fb.groupby('trading_volume', observed=True).mean()

,open,high,low,close,volume
trading_volume,,,,,
low,171.36,173.46,169.31,171.43,24547207.71
med,175.82,179.42,172.11,175.14,79072559.12
high,167.73,170.48,161.57,168.16,141924023.33


In [13]:
fb.groupby('trading_volume', observed=False)['close'].agg(['max', 'min', 'mean'])

,max,min,mean
trading_volume,,,
low,214.67,124.06,171.43
med,217.50,152.22,175.14
high,176.26,160.06,168.16


In [16]:
fb_agg = fb.groupby('trading_volume', observed=False).agg({
    'open': 'mean', 'high': ['min', 'max'],
    "low": ["min", "max"], "close": "mean"
})

fb_agg

open   high           low         close
                 mean    min    max    min    max   mean
trading_volume                                          
low            171.36 129.74 216.20 123.02 212.60 171.43
med            175.82 162.85 218.62 150.75 214.27 175.14
high           167.73 161.10 180.13 149.02 173.75 168.16

In [17]:
fb_agg.columns

MultiIndex([( 'open', 'mean'),
            ( 'high',  'min'),
            ( 'high',  'max'),
            (  'low',  'min'),
            (  'low',  'max'),
            ('close', 'mean')],
           )

In [18]:
fb_agg.columns = ['_'.join(col_agg) for col_agg in fb_agg.columns]

In [19]:
fb_agg

,open_mean,high_min,high_max,low_min,low_max,close_mean
trading_volume,,,,,,
low,171.36,129.74,216.20,123.02,212.60,171.43
med,175.82,162.85,218.62,150.75,214.27,175.14
high,167.73,161.10,180.13,149.02,173.75,168.16


In [24]:
weather.loc['2018-10'].query('datatype == "PRCP"').groupby(level=0).mean(numeric_only=True).head().squeeze()

date
2018-10-01    0.01
2018-10-02    2.23
2018-10-03   19.69
2018-10-04    0.32
2018-10-05    0.97
Name: value, dtype: float64

In [33]:
weather.query("datatype == 'PRCP'").groupby(['station_name', pd.Grouper(freq='QE')]).sum(numeric_only=True).unstack().sample(5, random_state=1)

value                                 
date                        2018-03-31 2018-06-30 2018-09-30 2018-12-31
station_name                                                           
WANTAGH 1.1 NNE, NY US          279.90     216.80     472.50     277.20
STATEN ISLAND 1.4 SE, NY US     379.40     295.30     438.80     409.90
SYOSSET 2.0 SSW, NY US          323.50     263.30     355.50     459.90
STAMFORD 4.2 S, CT US           338.00     272.10     424.70     390.00
WAYNE TWP 0.8 SSW, NJ US        246.20     295.30     620.90     422.00

In [39]:
weather.groupby('station_name').filter(
    lambda x: x.name.endswith('NY US')
).query("datatype == 'SNOW'").groupby("station_name").sum(numeric_only=True).squeeze()

station_name
ALBERTSON 0.2 SSE, NY US           1087.00
AMITYVILLE 0.1 WSW, NY US           434.00
AMITYVILLE 0.6 NNE, NY US          1072.00
ARMONK 0.3 SE, NY US               1504.00
BROOKLYN 3.1 NW, NY US              305.00
CENTERPORT 0.9 SW, NY US            799.00
CENTERPORT, NY US                  1333.00
ELMSFORD 0.8 SSW, NY US             863.00
FLORAL PARK 0.4 W, NY US           1015.00
HICKSVILLE 1.3 ENE, NY US           716.00
JACKSON HEIGHTS 0.3 WSW, NY US      107.00
JFK INTERNATIONAL AIRPORT, NY US    855.00
LA GUARDIA AIRPORT, NY US           892.00
LOCUST VALLEY 0.3 E, NY US            0.00
LYNBROOK 0.3 NW, NY US              325.00
MASSAPEQUA 0.9 SSW, NY US            41.00
MIDDLE VILLAGE 0.5 SW, NY US       1249.00
NEW HYDE PARK 1.6 NE, NY US           0.00
NEW YORK 8.8 N, NY US                 0.00
NORTH WANTAGH 0.4 WSW, NY US        471.00
NY CITY CENTRAL PARK, NY US        1007.00
PLAINEDGE 0.4 WSW, NY US            610.00
PLAINVIEW 0.4 ENE, NY US           1360.0

In [45]:
weather.query("datatype =='PRCP'").groupby(
    level=0
).mean(numeric_only=True).groupby(pd.Grouper(freq='ME')).sum(numeric_only=True).value.nlargest()

date
2018-11-30   210.59
2018-09-30   193.09
2018-08-31   192.45
2018-07-31   160.98
2018-02-28   158.11
Name: value, dtype: float64

In [52]:
weather.query("datatype == 'PRCP'").rename(dict(value='prcp'), axis='columns').groupby(
    level=0).mean(numeric_only=True).groupby(pd.Grouper(freq='ME')).transform('sum')['2018-01-28': '2018-02-03']

,prcp
date,
2018-01-28,69.31
2018-01-29,69.31
2018-01-30,69.31
2018-01-31,69.31
2018-02-01,158.11
2018-02-02,158.11
2018-02-03,158.11


In [55]:
weather.query("datatype == 'PRCP'").rename(dict(value='prcp'), axis='columns').groupby(
    level=0).mean(numeric_only=True).assign(
        total_prcp_in_month=lambda x: x.groupby(pd.Grouper(freq='ME')).transform('sum'),
        pct_monthly_prcp=lambda x: x['prcp'].div(x['total_prcp_in_month'])
    ).nlargest(5, columns='pct_monthly_prcp')

,prcp,total_prcp_in_month,pct_monthly_prcp
date,,,
2018-10-12,34.77,105.63,0.33
2018-01-13,21.66,69.31,0.31
2018-03-02,38.77,137.46,0.28
2018-04-16,39.34,140.57,0.28
2018-04-17,37.30,140.57,0.27


In [57]:
fb[['open', 'high', 'low', 'close']].transform(
    lambda x: (x - x.mean()).div(x.std())
).head()

,open,high,low,close
date,,,,
2018-01-02,0.32,0.41,0.41,0.50
2018-01-03,0.53,0.57,0.60,0.66
2018-01-04,0.68,0.65,0.74,0.64
2018-01-05,0.72,0.68,0.78,0.77
2018-01-08,0.80,0.79,0.85,0.84


#### Pivot tables and crosstabs

In [59]:
fb.pivot_table(columns='trading_volume', observed=True)

trading_volume,low,med,high
close,171.43,175.14,168.16
high,173.46,179.42,170.48
low,169.31,172.11,161.57
open,171.36,175.82,167.73
volume,24547207.71,79072559.12,141924023.33


In [62]:
fb.pivot_table(index='trading_volume', observed=True)

,close,high,low,open,volume
trading_volume,,,,,
low,171.43,173.46,169.31,171.36,24547207.71
med,175.14,179.42,172.11,175.82,79072559.12
high,168.16,170.48,161.57,167.73,141924023.33


In [64]:
weather.sample(2)

,datatype,station,value,station_name
date,,,,
2018-05-10,TMAX,GHCND:USW00014734,20.00,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US"
2018-06-15,PRCP,GHCND:US1NJMD0073,0.00,"CARTERET 0.6 WSW, NJ US"


In [69]:
weather.reset_index().pivot_table(index=['date', 'station_name', 'station'], columns='datatype', values='value',
                                  aggfunc='median').reset_index().tail()

datatype,date,station_name,station,AWND,DAPR,MDPR,PGTM,PRCP,SNOW,SNWD,...,WSF5,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
28740,2018-12-31,"WEST NYACK 1.3 WSW, NY US",GHCND:US1NYRL0005,NaN,NaN,NaN,NaN,0.30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28741,2018-12-31,"WESTCHESTER CO AIRPORT, NY US",GHCND:USW00094745,2.70,NaN,NaN,2212.00,24.40,NaN,NaN,...,11.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28742,2018-12-31,"WESTFIELD 0.6 NE, NJ US",GHCND:US1NJUN0014,NaN,NaN,NaN,NaN,0.30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28743,2018-12-31,"WOODBRIDGE TWP 1.1 ESE, NJ US",GHCND:US1NJMD0043,NaN,NaN,NaN,NaN,0.00,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28744,2018-12-31,"WOODBRIDGE TWP 1.1 NNE, NJ US",GHCND:US1NJMD0045,NaN,NaN,NaN,NaN,0.00,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
pd.crosstab(index=fb['trading_volume'], columns=fb.index.month, colnames=['month'])

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0


In [73]:
pd.crosstab(index=fb['trading_volume'], columns=fb.index.month, colnames=['month'],
            normalize='columns')

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,0.95,1.00,0.71,0.95,1.00,1.00,0.86,1.00,1.00,1.00,1.00,1.00
med,0.05,0.00,0.19,0.05,0.00,0.00,0.10,0.00,0.00,0.00,0.00,0.00
high,0.00,0.00,0.10,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00


In [74]:
pd.crosstab(index=fb['trading_volume'], columns=fb.index.month, colnames=['month'],
            values=fb['close'], aggfunc='mean')

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,185.24,180.27,177.07,163.29,182.93,195.27,201.92,177.49,164.38,154.19,141.64,137.16
med,179.37,NaN,164.76,174.16,NaN,NaN,194.28,NaN,NaN,NaN,NaN,NaN
high,NaN,NaN,164.11,NaN,NaN,NaN,176.26,NaN,NaN,NaN,NaN,NaN


In [77]:
snow_data = weather.query("datatype == 'SNOW'")
pd.crosstab(index=snow_data["station_name"],
            columns=snow_data.index.month,
            colnames=['month'], values=snow_data['value'], aggfunc=lambda x: (x > 0).sum(),
            margins=True, margins_name='Total observations of snow')

month,1,2,3,4,5,6,7,8,9,10,11,12,Total observations of snow
station_name,,,,,,,,,,,,,
"ALBERTSON 0.2 SSE, NY US",3.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,9
"AMITYVILLE 0.1 WSW, NY US",1.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3
"AMITYVILLE 0.6 NNE, NY US",3.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8
"ARMONK 0.3 SE, NY US",6.00,4.00,6.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,3.00,23
"BLOOMINGDALE 0.7 SSE, NJ US",2.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"WESTFIELD 0.6 NE, NJ US",3.00,0.00,4.00,1.00,0.00,NaN,0.00,0.00,0.00,NaN,1.00,NaN,9
"WOODBRIDGE TWP 1.1 ESE, NJ US",4.00,1.00,3.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,11
"WOODBRIDGE TWP 1.1 NNE, NJ US",2.00,1.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,7
